In [1]:
import os
from time import sleep
import json
from tqdm import tqdm
tqdm.pandas(desc='pandas')

import pandas as pd
import numpy as np
from dask import dataframe as dd

from geopy.geocoders import Nominatim
geoEncoder = Nominatim(user_agent='spanish-cities')

def backup(dic, path):
    with open(path,'w') as f:
        json.dump(obj=dic, fp=f, indent=4)

def restore(path):
    with open(path) as f:
        return json.load(f)
        
def distance(coords1: tuple, coords2: tuple):
    return np.sqrt((coords1[0] - coords2[0])**2  +  (coords1[1] - coords2[1])**2)

C:\Users\astro\AppData\Roaming\Python\Python39\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## city associations

### get glovo city coords

In [2]:
df = pd.read_excel('glovo_city_codes.xlsx', index_col=0)

if not os.path.exists('glovoCityCoords.json'):
    glovoCityCoords = dict.fromkeys(list(df.city_name), np.nan)  

    for city in tqdm(glovoCityCoords.keys()):
        sleep(1)
        try:    
            geoInfo = geoEncoder.geocode(f'{city}, Spain').raw
            glovoCityCoords[city] = float(geoInfo['lat']), float(geoInfo['lon'])
        except:
            glovoCityCoords[city] = np.nan, np.nan

    backup(glovoCityCoords, 'glovoCityCoords.json')

else:
    glovoCityCoords = restore('glovoCityCoords.json')

glovoCityCoords

{'Aguadulce': [37.2530195, -4.9906167],
 'Albacete': [38.9950921, -1.8559154],
 'Alicante': [38.3436365, -0.4881708],
 'Algeciras': [36.1311725, -5.4473991],
 'Almeria': [36.8414197, -2.4628135],
 'Alcantarilla ': [37.9680342, -1.214954],
 'Avila': [40.656478, -4.7002172],
 'Barcelona': [41.3828939, 2.1774322],
 'Bilbao': [43.2630018, -2.9350039],
 'Badajoz': [38.88964355, -6.980142450723942],
 'Barakaldo': [43.29548, -2.9900933],
 'Benidorm': [38.5406255, -0.1290929],
 'Brunete': [40.4050499, -3.9979912],
 'Cartagena': [37.6019353, -0.9841152],
 'Cadiz': [36.5315575993944, -6.280563331226297],
 'Colmenar Viejo': [40.6587726, -3.7659722],
 'Castelldefels': [41.2861022, 1.9824173],
 'Castellón de la Plana': [39.9860347, -0.0377354],
 'Sant Cugat del Valles': [41.4728432, 2.0817809],
 'Donosita': [43.39907934659774, -1.975314531094155],
 'Elche': [38.2653307, -0.6988391],
 'Fuenlabrada': [40.282476, -3.7923422],
 'San Fernando': [36.4643934, -6.198203],
 'Gava': [41.3050933, 2.0063126],


In [3]:
# check how many glovo cities have no coordinates
[city for city, coords in glovoCityCoords.items() if np.isnan(coords[0])]

[]

In [4]:
# # lookup missings manually
# glovoCityCoords['Cadiz'] = [36.5315575993944, -6.280563331226297]
# glovoCityCoords['Donosita'] = [43.39907934659774, -1.975314531094155]
# glovoCityCoords['Valencia'] = [39.47424750070927, -0.37628868297826856]

# # del glovoCityCoords['Foligno']


# backup(glovoCityCoords, 'glovoCityCoords.json')

### find closest glovo city for all census towns

In [5]:
municipalities = pd.read_excel('../aux_data/01_census/data/provincial and municipal codes.xlsx', usecols=['CODAUTO', 'NOMBRE'])
municipalities

,CODAUTO,NOMBRE
0,16,Alegría-Dulantzi
1,16,Amurrio
2,16,Aramaio
3,16,Artziniega
4,16,Armiñón
...,...,...
8126,2,"Zaida, La"
8127,2,Zaragoza
8128,2,Zuera
8129,18,Ceuta


In [6]:
autoCommCodes = pd.read_excel('autonomousCommunitiesCodes.xlsx', usecols=['CODAUTO', 'Comunidad Autónoma']).drop_duplicates().reset_index(drop=True)
autoCommCodes

,CODAUTO,Comunidad Autónoma
0,1,Andalucía
1,2,Aragón
2,3,"Asturias, Principado de"
3,4,"Balears, Illes"
4,5,Canarias
...,...,...
14,15,"Navarra, Comunidad Foral de"
15,16,País Vasco
16,17,"Rioja, La"
17,18,Ceuta


In [7]:
# association table
ass = pd.merge(municipalities, autoCommCodes, on='CODAUTO', how='inner')
ass = (
    ass
    .drop(columns=['CODAUTO'])
    .rename(columns={
        'Comunidad Autónoma': 'autonomous_community', 
        'NOMBRE': 'municipality',
        })
)
ass

,municipality,autonomous_community
0,Alegría-Dulantzi,País Vasco
1,Amurrio,País Vasco
2,Aramaio,País Vasco
3,Artziniega,País Vasco
4,Armiñón,País Vasco
...,...,...
8126,Villaescusa,Cantabria
8127,Villafufre,Cantabria
8128,Voto,Cantabria
8129,Ceuta,Ceuta


In [8]:
def getCoords(location: str):
    sleep(0.8)
    try:    
        geoInfo = geoEncoder.geocode(f'{location}, Spain').raw
        return float(geoInfo['lat']), float(geoInfo['lon'])
    except:
        return np.nan, np.nan

def retrieveCoords(city: str, censusMunicipalityCoords: dict):
    if (city in censusMunicipalityCoords) & (not np.isnan(censusMunicipalityCoords[city][0])):
        return censusMunicipalityCoords[city]
    else:
        return getCoords(city)
        

def findClosestGlovoCity(testCoords: tuple, glovoCityCoords: dict):
    '''
    finds closest glovo-city in a given radius. 
    trialled radiusses:
    - 10km/0.15 deg: 95% of municipalities have no glovo-city -> 200 municipalities for 70 cities: 3 municipalities/city
    - 20km/0.30 deg: 75% of municipalities have no glovo-city - > 2000 municipalities for 70 cities: 30 municipalities/city
      (despite this, 1.7m of 2m census observations still have a glovo-city, because the areas surrounding glovo cities are the most populated, thus most represented in census)
    '''
    theFavourite = ('', np.inf)

    for city, coords in glovoCityCoords.items():
        d = distance(testCoords, coords)

        if d < theFavourite[1] and d < 0.15:     
            theFavourite = (city, d)

    return theFavourite[0]

In [9]:
censusMunicipalityCoords = dict.fromkeys(list(municipalities.NOMBRE), np.nan) if not os.path.exists('censusMunicipalityCoords.json') else restore('censusMunicipalityCoords.json')

ass['municipality_coords'] = ass.municipality.progress_apply(retrieveCoords, censusMunicipalityCoords=censusMunicipalityCoords)
ass['closest_glovo_city'] = ass.municipality_coords.progress_apply(findClosestGlovoCity, glovoCityCoords=glovoCityCoords)
ass



pandas: 100%|██████████| 8131/8131 [00:01<00:00, 5229.48it/s]


,municipality,autonomous_community,municipality_coords,closest_glovo_city
0,Alegría-Dulantzi,País Vasco,"[42.8424145, -2.512674]",
1,Amurrio,País Vasco,"[43.0525066, -3.000896]",
2,Aramaio,País Vasco,"[43.035206099999996, -2.585761508234]",
3,Artziniega,País Vasco,"[43.1210566, -3.1286742]",
4,Armiñón,País Vasco,"[42.722587, -2.8722115]",
...,...,...,...,...
8126,Villaescusa,Cantabria,"[41.2055283, -5.4639875]",
8127,Villafufre,Cantabria,"[43.266473, -3.892813]",
8128,Voto,Cantabria,"[43.3467008, -3.510960548422662]",
8129,Ceuta,Ceuta,"[35.89442195, -5.355817352394269]",


In [10]:
ass.isnull().sum()

municipality            0
autonomous_community    0
municipality_coords     0
closest_glovo_city      0
dtype: int64

In [11]:
ass[ass.closest_glovo_city == ''].reset_index()

,index,municipality,autonomous_community,municipality_coords,closest_glovo_city
0,0,Alegría-Dulantzi,País Vasco,"[42.8424145, -2.512674]",
1,1,Amurrio,País Vasco,"[43.0525066, -3.000896]",
2,2,Aramaio,País Vasco,"[43.035206099999996, -2.585761508234]",
3,3,Artziniega,País Vasco,"[43.1210566, -3.1286742]",
4,4,Armiñón,País Vasco,"[42.722587, -2.8722115]",
...,...,...,...,...,...
7293,8126,Villaescusa,Cantabria,"[41.2055283, -5.4639875]",
7294,8127,Villafufre,Cantabria,"[43.266473, -3.892813]",
7295,8128,Voto,Cantabria,"[43.3467008, -3.510960548422662]",
7296,8129,Ceuta,Ceuta,"[35.89442195, -5.355817352394269]",


In [12]:
ass = ass[ass.closest_glovo_city != ''].reset_index(drop=True)
ass

,municipality,autonomous_community,municipality_coords,closest_glovo_city
0,Arratzua-Ubarrundia,País Vasco,"[42.91823515, -2.590652453198305]",Vitoria
1,Zigoitia,País Vasco,"[42.9568048, -2.7350578]",Vitoria
2,Elburgo/Burgelu,País Vasco,"[42.8495927, -2.5448025]",Vitoria
3,Laudio/Llodio,País Vasco,"[43.1430692, -2.9624876]",Bilbao
4,Okondo,País Vasco,"[43.1547427, -3.0267479]",Bilbao
...,...,...,...,...
828,Medio Cudeyo,Cantabria,"[43.380690200000004, -3.7624661487619235]",Santander
829,Penagos,Cantabria,"[43.3413728, -3.8068404]",Santander
830,Ribamontán al Mar,Cantabria,"[43.460752, -3.689898374785143]",Santander
831,Santa Cruz de Bezana,Cantabria,"[43.4429266, -3.9048376]",Santander


In [13]:
ass.to_excel('association_table.xlsx')

## census 

### adding associated glovo-city

In [14]:
census = dd.read_parquet('../aux_data/01_census/data/locationSubset/').compute()
census.head()

,province code,Municipality code or size,Hole identifier,Final number of the person inside the hole,Person lift factor,Birth month,Year of birth,Age,Sex,Country code of nationality,...,core type,core size,Number of children,Number of common children of the kernel,Large family indicator,Type of partner (de facto or de jure),"Type of couple (same sex, different sex)",Age difference between male and female core,locationCode,location
2,1.0,59.0,65.0,1.0,12.585368,1.0,1915.0,96.0,1.0,108.0,...,1.0,1.0,0.0,0.0,1.0,1.0,1.0,6.0,01059,Vitoria-Gasteiz
3,1.0,59.0,65.0,2.0,12.585368,6.0,1923.0,88.0,6.0,108.0,...,1.0,1.0,0.0,0.0,1.0,1.0,1.0,6.0,01059,Vitoria-Gasteiz
4,1.0,59.0,138.0,1.0,14.037818,1.0,1917.0,94.0,1.0,108.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01059,Vitoria-Gasteiz
5,1.0,59.0,202.0,1.0,41.444004,1.0,1918.0,93.0,1.0,108.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01059,Vitoria-Gasteiz
6,1.0,59.0,202.0,2.0,41.444004,7.0,1964.0,47.0,1.0,341.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01059,Vitoria-Gasteiz


In [15]:
len(census)

2085622

In [16]:
ass = pd.read_excel('association_table.xlsx', index_col=0).drop(columns='municipality_coords')
ass

,municipality,autonomous_community,closest_glovo_city
0,Arratzua-Ubarrundia,País Vasco,Vitoria
1,Zigoitia,País Vasco,Vitoria
2,Elburgo/Burgelu,País Vasco,Vitoria
3,Laudio/Llodio,País Vasco,Bilbao
4,Okondo,País Vasco,Bilbao
...,...,...,...
828,Medio Cudeyo,Cantabria,Santander
829,Penagos,Cantabria,Santander
830,Ribamontán al Mar,Cantabria,Santander
831,Santa Cruz de Bezana,Cantabria,Santander


In [17]:
ass[ass.municipality == 'Manresa']

,municipality,autonomous_community,closest_glovo_city


In [18]:
census = pd.merge(census, ass, left_on='location', right_on='municipality', how='inner')
census

,province code,Municipality code or size,Hole identifier,Final number of the person inside the hole,Person lift factor,Birth month,Year of birth,Age,Sex,Country code of nationality,...,Number of common children of the kernel,Large family indicator,Type of partner (de facto or de jure),"Type of couple (same sex, different sex)",Age difference between male and female core,locationCode,location,municipality,autonomous_community,closest_glovo_city
0,1.0,59.0,65.0,1.0,12.585368,1.0,1915.0,96.0,1.0,108.0,...,0.0,1.0,1.0,1.0,6.0,01059,Vitoria-Gasteiz,Vitoria-Gasteiz,País Vasco,Vitoria
1,1.0,59.0,65.0,2.0,12.585368,6.0,1923.0,88.0,6.0,108.0,...,0.0,1.0,1.0,1.0,6.0,01059,Vitoria-Gasteiz,Vitoria-Gasteiz,País Vasco,Vitoria
2,1.0,59.0,138.0,1.0,14.037818,1.0,1917.0,94.0,1.0,108.0,...,NaN,NaN,NaN,NaN,NaN,01059,Vitoria-Gasteiz,Vitoria-Gasteiz,País Vasco,Vitoria
3,1.0,59.0,202.0,1.0,41.444004,1.0,1918.0,93.0,1.0,108.0,...,NaN,NaN,NaN,NaN,NaN,01059,Vitoria-Gasteiz,Vitoria-Gasteiz,País Vasco,Vitoria
4,1.0,59.0,202.0,2.0,41.444004,7.0,1964.0,47.0,1.0,341.0,...,NaN,NaN,NaN,NaN,NaN,01059,Vitoria-Gasteiz,Vitoria-Gasteiz,País Vasco,Vitoria
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1574703,50.0,297.0,1621532.0,3.0,12.520465,1.0,1974.0,37.0,6.0,108.0,...,0.0,1.0,4.0,4.0,8.0,50297,Zaragoza,Zaragoza,Aragón,Zaragoza
1574704,50.0,297.0,1621643.0,1.0,12.517144,12.0,2010.0,0.0,6.0,108.0,...,1.0,1.0,1.0,1.0,3.0,50297,Zaragoza,Zaragoza,Aragón,Zaragoza
1574705,50.0,297.0,1621643.0,2.0,12.517144,12.0,2001.0,9.0,1.0,108.0,...,1.0,1.0,1.0,1.0,3.0,50297,Zaragoza,Zaragoza,Aragón,Zaragoza
1574706,50.0,297.0,1621643.0,3.0,12.517144,7.0,1968.0,43.0,6.0,108.0,...,1.0,1.0,1.0,1.0,3.0,50297,Zaragoza,Zaragoza,Aragón,Zaragoza


In [19]:
census[census.closest_glovo_city.isna()]

,province code,Municipality code or size,Hole identifier,Final number of the person inside the hole,Person lift factor,Birth month,Year of birth,Age,Sex,Country code of nationality,...,Number of common children of the kernel,Large family indicator,Type of partner (de facto or de jure),"Type of couple (same sex, different sex)",Age difference between male and female core,locationCode,location,municipality,autonomous_community,closest_glovo_city


### create each glovo-city-region's share of foreigners

In [20]:
colsOfInterest = ['closest_glovo_city', 'municipality', 'autonomous_community', 'Age', 'Sex', 'Country code of nationality', 'Country of birth code']
byCity = census.groupby('closest_glovo_city')[colsOfInterest]
byCity.get_group('Vitoria')

,closest_glovo_city,municipality,autonomous_community,Age,Sex,Country code of nationality,Country of birth code
0,Vitoria,Vitoria-Gasteiz,País Vasco,96.0,1.0,108.0,108.0
1,Vitoria,Vitoria-Gasteiz,País Vasco,88.0,6.0,108.0,108.0
2,Vitoria,Vitoria-Gasteiz,País Vasco,94.0,1.0,108.0,108.0
3,Vitoria,Vitoria-Gasteiz,País Vasco,93.0,1.0,108.0,108.0
4,Vitoria,Vitoria-Gasteiz,País Vasco,47.0,1.0,341.0,341.0
...,...,...,...,...,...,...,...
16654,Vitoria,Vitoria-Gasteiz,País Vasco,36.0,1.0,228.0,228.0
16655,Vitoria,Vitoria-Gasteiz,País Vasco,3.0,6.0,228.0,228.0
16656,Vitoria,Vitoria-Gasteiz,País Vasco,4.0,1.0,228.0,228.0
16657,Vitoria,Vitoria-Gasteiz,País Vasco,2.0,6.0,228.0,108.0


In [21]:
topCountryCodes = list(census['Country of birth code'].value_counts(dropna=False).index)[:20]
topCountryCodes

[108.0,
 345.0,
 228.0,
 343.0,
 128.0,
 340.0,
 348.0,
 110.0,
 341.0,
 126.0,
 351.0,
 326.0,
 315.0,
 125.0,
 342.0,
 407.0,
 350.0,
 344.0,
 347.0,
 115.0]

In [22]:
countryCodeKeyDF = pd.read_excel('country_codes_key.xlsx')
countryCodeKeyDF

,code,country
0,102,Austria
1,103,Bélgica
2,104,Bulgaria
3,106,Chipre
4,107,Dinamarca
...,...,...
196,514,Kiribati
197,515,Nauru
198,516,Palaos
199,517,Timor Oriental


In [23]:
collapsedAux = pd.DataFrame()

for glovoCityGroupName, glovoCityGroup in byCity.__iter__():      # for each group of municipalities assigned to one glovo-city, take all the observations of those (several) municipalities and calculate the share of different kinds of foreigners

    row = pd.Series(name=glovoCityGroupName, dtype=float) 
    
    row['total_observations'] = len(glovoCityGroup) 
    row['mean_age'] = 0.            # initialise as float! otherwise it will get downcast to int, and everything will be zero ://////
    row['mean_female_share'] = 0.
    for code in topCountryCodes:
            countryName = countryCodeKeyDF[countryCodeKeyDF.code == int(code)].country.values[0].replace(' ', '_').lower()
            row[f'share_{countryName}'] = 0.


    
    byMunicipality = glovoCityGroup.groupby('municipality')
    for municipality, municipalityGroup in byMunicipality.__iter__():
        
        weight = len(municipalityGroup) / row['total_observations']

        row['mean_age'] += municipalityGroup.Age.mean() * weight
        row['mean_female_share'] += municipalityGroup.Sex.replace({1.0: 0, 6.0: 1}).mean() * weight
        
        
        for code in topCountryCodes:
            share = len(municipalityGroup[municipalityGroup['Country code of nationality'] == code]) / len(municipalityGroup)

            countryName = countryCodeKeyDF[countryCodeKeyDF.code == int(code)].country.values[0].replace(' ', '_').lower()
            row[f'share_{countryName}'] += share * weight

    

    collapsedAux = collapsedAux.append(row)

collapsedAux = collapsedAux.reset_index().rename(columns={'index': 'closest_glovo_city'})

collapsedAux



,closest_glovo_city,total_observations,mean_age,mean_female_share,share_españa,share_ecuador,share_marruecos,share_colombia,share_rumanía,share_argentina,...,share_venezuela,share_república_dominicana,share_cuba,share_reino_unido,share_brasil,share_china,share_uruguay,share_chile,share_paraguay,share_italia
0,A Coruña,23636.0,43.946691,0.528685,0.967084,0.000381,0.000804,0.002919,0.000296,0.002750,...,0.002115,0.001904,0.001946,0.000931,0.003215,0.000465,0.003046,0.000465,0.000169,0.001777
1,Albacete,11912.0,39.921592,0.519476,0.966924,0.001763,0.002015,0.007220,0.004701,0.001259,...,0.000084,0.000336,0.000336,0.000168,0.000420,0.000923,0.000084,0.000588,0.001175,0.000588
2,Alcala de Henares,17089.0,39.858740,0.508573,0.915384,0.003218,0.003043,0.006729,0.033530,0.001404,...,0.000410,0.001990,0.000468,0.000234,0.001404,0.001404,0.000351,0.001287,0.000176,0.001346
3,Alcantarilla,2902.0,36.694349,0.502412,0.936940,0.015851,0.009304,0.003790,0.005513,0.002068,...,0.000345,0.000000,0.000345,0.000345,0.001378,0.001034,0.000345,0.001034,0.000000,0.001378
4,Algeciras,10335.0,39.736526,0.516014,0.966812,0.000484,0.008128,0.000774,0.001258,0.001742,...,0.000097,0.000290,0.000000,0.007644,0.000774,0.000000,0.000000,0.000581,0.000484,0.001645
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,Valladolid,25382.0,44.592349,0.526712,0.966590,0.002915,0.002600,0.004137,0.002206,0.000670,...,0.001024,0.002718,0.000827,0.000158,0.001576,0.000867,0.000039,0.000000,0.000473,0.000473
68,Vigo,25404.0,43.782436,0.526610,0.963116,0.000236,0.000197,0.002677,0.002047,0.003188,...,0.002992,0.000866,0.001693,0.000590,0.003858,0.000276,0.003188,0.000354,0.000630,0.001378
69,Vitoria,16659.0,42.676091,0.512816,0.940693,0.003842,0.006483,0.008524,0.003181,0.001020,...,0.000900,0.001621,0.001381,0.000480,0.002221,0.000360,0.000120,0.001141,0.003241,0.001020
70,Xativa,1822.0,41.416026,0.524698,0.959385,0.005488,0.002744,0.001098,0.007135,0.000000,...,0.000000,0.000000,0.000000,0.002195,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [24]:
# add autonomous community to each glovo-city by looking up the most popular AC in the municipalities assigned to that glovo city
for glovoCity in collapsedAux.closest_glovo_city:
    # for each glovo city, the dominant autonomous community is those with the most municipalities for which this glovo city is the closest
    collapsedAux.loc[collapsedAux.closest_glovo_city == glovoCity, 'autonomous_community'] = ass[['closest_glovo_city', 'autonomous_community']].groupby('closest_glovo_city').get_group(glovoCity).value_counts('autonomous_community').index[0]
collapsedAux

,closest_glovo_city,total_observations,mean_age,mean_female_share,share_españa,share_ecuador,share_marruecos,share_colombia,share_rumanía,share_argentina,...,share_república_dominicana,share_cuba,share_reino_unido,share_brasil,share_china,share_uruguay,share_chile,share_paraguay,share_italia,autonomous_community
0,A Coruña,23636.0,43.946691,0.528685,0.967084,0.000381,0.000804,0.002919,0.000296,0.002750,...,0.001904,0.001946,0.000931,0.003215,0.000465,0.003046,0.000465,0.000169,0.001777,Galicia
1,Albacete,11912.0,39.921592,0.519476,0.966924,0.001763,0.002015,0.007220,0.004701,0.001259,...,0.000336,0.000336,0.000168,0.000420,0.000923,0.000084,0.000588,0.001175,0.000588,Castilla-La Mancha
2,Alcala de Henares,17089.0,39.858740,0.508573,0.915384,0.003218,0.003043,0.006729,0.033530,0.001404,...,0.001990,0.000468,0.000234,0.001404,0.001404,0.000351,0.001287,0.000176,0.001346,"Madrid, Comunidad de"
3,Alcantarilla,2902.0,36.694349,0.502412,0.936940,0.015851,0.009304,0.003790,0.005513,0.002068,...,0.000000,0.000345,0.000345,0.001378,0.001034,0.000345,0.001034,0.000000,0.001378,"Murcia, Región de"
4,Algeciras,10335.0,39.736526,0.516014,0.966812,0.000484,0.008128,0.000774,0.001258,0.001742,...,0.000290,0.000000,0.007644,0.000774,0.000000,0.000000,0.000581,0.000484,0.001645,Andalucía
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,Valladolid,25382.0,44.592349,0.526712,0.966590,0.002915,0.002600,0.004137,0.002206,0.000670,...,0.002718,0.000827,0.000158,0.001576,0.000867,0.000039,0.000000,0.000473,0.000473,Castilla y León
68,Vigo,25404.0,43.782436,0.526610,0.963116,0.000236,0.000197,0.002677,0.002047,0.003188,...,0.000866,0.001693,0.000590,0.003858,0.000276,0.003188,0.000354,0.000630,0.001378,Galicia
69,Vitoria,16659.0,42.676091,0.512816,0.940693,0.003842,0.006483,0.008524,0.003181,0.001020,...,0.001621,0.001381,0.000480,0.002221,0.000360,0.000120,0.001141,0.003241,0.001020,País Vasco
70,Xativa,1822.0,41.416026,0.524698,0.959385,0.005488,0.002744,0.001098,0.007135,0.000000,...,0.000000,0.000000,0.002195,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Comunitat Valenciana


In [25]:
collapsedAux.autonomous_community.value_counts(dropna=False)

Andalucía                      14
Madrid, Comunidad de           11
Cataluña                       10
Comunitat Valenciana            7
Galicia                         5
Castilla y León                 5
País Vasco                      5
Murcia, Región de               4
Asturias, Principado de         2
Balears, Illes                  2
Canarias                        2
Castilla-La Mancha              1
Extremadura                     1
Navarra, Comunidad Foral de     1
Cantabria                       1
Aragón                          1
Name: autonomous_community, dtype: int64

In [26]:
sorted(collapsedAux.autonomous_community.unique())

['Andalucía',
 'Aragón',
 'Asturias, Principado de',
 'Balears, Illes',
 'Canarias',
 'Cantabria',
 'Castilla y León',
 'Castilla-La Mancha',
 'Cataluña',
 'Comunitat Valenciana',
 'Extremadura',
 'Galicia',
 'Madrid, Comunidad de',
 'Murcia, Región de',
 'Navarra, Comunidad Foral de',
 'País Vasco']

In [27]:
sorted(ass.autonomous_community.unique())
# -> Ceuta & Melilla are missing (are independent cities anyway)

['Andalucía',
 'Aragón',
 'Asturias, Principado de',
 'Balears, Illes',
 'Canarias',
 'Cantabria',
 'Castilla y León',
 'Castilla-La Mancha',
 'Cataluña',
 'Comunitat Valenciana',
 'Extremadura',
 'Galicia',
 'Madrid, Comunidad de',
 'Murcia, Región de',
 'Navarra, Comunidad Foral de',
 'País Vasco']

In [28]:
collapsedAux.to_excel('collapsed_aux.xlsx')

## ESS  

### select years and countries

In [29]:
ESS = pd.read_csv('../aux_data/03_xenophobia/ESS/data/columnSubset_normalised_indeces.csv', index_col=0, low_memory=False)
ESS = ESS[
    (ESS.cntry == 'ES')  
    & (ESS.year >= 2010) 
    & ~(ESS.region_decoded.isin(['Ciudad Autónoma de Ceuta', 'Ciudad Autónoma de Melilla'])) # only excludes 34 observations
    ]
ESS.reset_index(drop=True)

,imsmetn,imdfetn,impcntr,imbgeco,imueclt,imwbcnt,cntry,idno,year,anctry1,...,region_decoded,imsmetn_norm,imdfetn_norm,impcntr_norm,imbgeco_norm,imueclt_norm,imwbcnt_norm,index_antiimmigration,index_xenophobia,index_all
0,3.0,3.0,3.0,8.0,8.0,8.0,ES,1.0,2010,NaN,...,Comunidad de Madrid,0.452481,0.210653,0.131665,0.058908,-0.985295,0.063442,0.213427,-0.460927,-0.011358
1,4.0,4.0,4.0,0.0,0.0,0.0,ES,2.0,2010,NaN,...,País Vasco,1.176814,0.939987,0.839108,0.494751,2.108017,0.493011,0.862665,1.300514,1.008615
2,2.0,2.0,2.0,6.0,8.0,5.0,ES,3.0,2010,NaN,...,Castilla y León,-0.271852,-0.518681,-0.575777,0.167869,-0.985295,0.224530,-0.299611,-0.380383,-0.326535
3,4.0,4.0,4.0,0.0,0.0,5.0,ES,6.0,2010,NaN,...,Comunidad Valenciana,1.176814,0.939987,0.839108,0.494751,2.108017,0.224530,0.862665,1.166273,0.963868
4,2.0,2.0,2.0,6.0,6.0,6.0,ES,8.0,2010,NaN,...,Galicia,-0.271852,-0.518681,-0.575777,0.167869,-0.211967,0.170834,-0.299611,-0.020567,-0.206596
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9286,2.0,2.0,3.0,5.0,6.0,1.0,ES,69763.0,2018,13070.0,...,Cataluña,-0.271852,-0.518681,0.131665,0.222349,-0.211967,0.439315,-0.109130,0.113674,-0.034862
9287,3.0,3.0,3.0,6.0,7.0,8.0,ES,69803.0,2018,13070.0,...,Aragón,0.452481,0.210653,0.131665,0.167869,-0.598631,0.063442,0.240667,-0.267595,0.071246
9288,2.0,2.0,2.0,8.0,9.0,9.0,ES,69837.0,2018,61070.0,...,Comunidad Valenciana,-0.271852,-0.518681,-0.575777,0.058908,-1.371959,0.009745,-0.326851,-0.681107,-0.444936
9289,8.0,8.0,8.0,88.0,6.0,88.0,ES,69939.0,2018,13070.0,...,País Vasco,4.074146,3.857324,3.668878,-4.299521,-0.211967,-4.232249,1.825207,-2.222108,0.476102


In [30]:
ESS.year.value_counts()

2016    1951
2014    1916
2012    1883
2010    1880
2018    1661
Name: year, dtype: int64

In [31]:
ESS.region_decoded.value_counts()

Andalucía                     1862
Comunidad de Madrid           1312
Cataluña                      1268
Comunidad Valenciana           924
Galicia                        622
Castilla y León                544
Castilla-La Mancha             448
País Vasco                     427
Canarias                       354
Aragón                         308
Extremadura                    252
Región de Murcia               248
Principado de Asturias         228
Illes Balears                  180
Cantabria                      130
Comunidad Foral de Navarra     126
La Rioja                        58
Name: region_decoded, dtype: int64

### fuzzy matching

In [32]:
sorted(ESS.region_decoded.unique())

['Andalucía',
 'Aragón',
 'Canarias',
 'Cantabria',
 'Castilla y León',
 'Castilla-La Mancha',
 'Cataluña',
 'Comunidad Foral de Navarra',
 'Comunidad Valenciana',
 'Comunidad de Madrid',
 'Extremadura',
 'Galicia',
 'Illes Balears',
 'La Rioja',
 'País Vasco',
 'Principado de Asturias',
 'Región de Murcia']

In [33]:
sorted(collapsedAux.autonomous_community.unique())

['Andalucía',
 'Aragón',
 'Asturias, Principado de',
 'Balears, Illes',
 'Canarias',
 'Cantabria',
 'Castilla y León',
 'Castilla-La Mancha',
 'Cataluña',
 'Comunitat Valenciana',
 'Extremadura',
 'Galicia',
 'Madrid, Comunidad de',
 'Murcia, Región de',
 'Navarra, Comunidad Foral de',
 'País Vasco']

In [34]:
collapsedESS = ESS[['region_decoded', 'index_antiimmigration', 'index_xenophobia', 'index_all']].groupby('region_decoded').mean().reset_index()
collapsedESS

,region_decoded,index_antiimmigration,index_xenophobia,index_all
0,Andalucía,-0.024655,-0.132005,-0.048291
1,Aragón,0.049386,-0.133809,-0.005413
2,Canarias,-0.008962,-0.082150,-0.029668
3,Cantabria,0.023196,-0.185491,-0.038931
4,Castilla y León,0.091483,-0.113672,0.030717
...,...,...,...,...
12,Illes Balears,-0.075930,-0.134542,-0.092726
13,La Rioja,0.004147,-0.196317,-0.053123
14,País Vasco,-0.074517,-0.209440,-0.107988
15,Principado de Asturias,0.105272,-0.128852,0.051152


In [35]:
from fuzzywuzzy import process

process.extract('Principado de Asturias', choices=collapsedAux.autonomous_community.unique())


[('Asturias, Principado de', 95),
 ('Murcia, Región de', 56),
 ('Canarias', 56),
 ('Galicia', 51),
 ('Cataluña', 51)]

In [36]:
process.extractOne('Principado de Asturias', choices=collapsedAux.autonomous_community.unique(), score_cutoff=90)

('Asturias, Principado de', 95)

In [37]:
collapsedESS['autonomous_community'] = collapsedESS.region_decoded.apply(lambda region: process.extractOne(region, choices=collapsedAux.autonomous_community.unique(), score_cutoff=90))
pd.options.display.min_rows = 50
collapsedESS

,region_decoded,index_antiimmigration,index_xenophobia,index_all,autonomous_community
0,Andalucía,-0.024655,-0.132005,-0.048291,"(Andalucía, 100)"
1,Aragón,0.049386,-0.133809,-0.005413,"(Aragón, 100)"
2,Canarias,-0.008962,-0.082150,-0.029668,"(Canarias, 100)"
3,Cantabria,0.023196,-0.185491,-0.038931,"(Cantabria, 100)"
4,Castilla y León,0.091483,-0.113672,0.030717,"(Castilla y León, 100)"
5,Castilla-La Mancha,0.023379,-0.067050,0.004133,"(Castilla-La Mancha, 100)"
6,Cataluña,-0.040758,-0.129260,-0.065620,"(Cataluña, 100)"
7,Comunidad Foral de Navarra,-0.156991,-0.130415,-0.143776,"(Navarra, Comunidad Foral de, 95)"
8,Comunidad Valenciana,-0.060331,-0.146607,-0.080495,"(Comunitat Valenciana, 90)"
9,Comunidad de Madrid,-0.072231,-0.245129,-0.121979,"(Madrid, Comunidad de, 95)"


In [40]:
ass.autonomous_community.unique()

array(['País Vasco', 'Castilla-La Mancha', 'Comunitat Valenciana',
       'Andalucía', 'Castilla y León', 'Extremadura', 'Balears, Illes',
       'Cataluña', 'Galicia', 'Aragón', 'Madrid, Comunidad de',
       'Murcia, Región de', 'Navarra, Comunidad Foral de',
       'Asturias, Principado de', 'Canarias', 'Cantabria'], dtype=object)

In [44]:
def fuzzyMatch(region, choices):
    try:
        return process.extractOne(region, choices=choices, score_cutoff=90)[0]
    except:
        return np.nan
    
collapsedESS['autonomous_community'] = collapsedESS.region_decoded.apply(lambda region: fuzzyMatch(region, choices=collapsedAux.autonomous_community.unique()))
collapsedESS = collapsedESS.dropna(subset=['autonomous_community'], axis='rows')
collapsedESS

,region_decoded,index_antiimmigration,index_xenophobia,index_all,autonomous_community
0,Andalucía,-0.024655,-0.132005,-0.048291,Andalucía
1,Aragón,0.049386,-0.133809,-0.005413,Aragón
2,Canarias,-0.008962,-0.082150,-0.029668,Canarias
3,Cantabria,0.023196,-0.185491,-0.038931,Cantabria
4,Castilla y León,0.091483,-0.113672,0.030717,Castilla y León
5,Castilla-La Mancha,0.023379,-0.067050,0.004133,Castilla-La Mancha
6,Cataluña,-0.040758,-0.129260,-0.065620,Cataluña
7,Comunidad Foral de Navarra,-0.156991,-0.130415,-0.143776,"Navarra, Comunidad Foral de"
8,Comunidad Valenciana,-0.060331,-0.146607,-0.080495,Comunitat Valenciana
9,Comunidad de Madrid,-0.072231,-0.245129,-0.121979,"Madrid, Comunidad de"


### merging ESS to aux

In [45]:
collapsedAux = pd.merge(collapsedAux, collapsedESS, on='autonomous_community', how='inner').drop(columns=['region_decoded'])
collapsedAux

,closest_glovo_city,total_observations,mean_age,mean_female_share,share_españa,share_ecuador,share_marruecos,share_colombia,share_rumanía,share_argentina,...,share_brasil,share_china,share_uruguay,share_chile,share_paraguay,share_italia,autonomous_community,index_antiimmigration,index_xenophobia,index_all
0,A Coruña,23636.0,43.946691,0.528685,0.967084,0.000381,0.000804,0.002919,0.000296,0.002750,...,0.003215,0.000465,0.003046,0.000465,0.000169,0.001777,Galicia,-0.103921,-0.140070,-0.100273
1,Ourense,6888.0,45.570412,0.539925,0.970383,0.000145,0.001597,0.004646,0.000726,0.001887,...,0.002613,0.000290,0.000145,0.000000,0.000871,0.000581,Galicia,-0.103921,-0.140070,-0.100273
2,Pontevedra,5121.0,41.683460,0.534075,0.968366,0.000586,0.003320,0.003710,0.000781,0.001562,...,0.003905,0.002343,0.001172,0.000000,0.000000,0.000391,Galicia,-0.103921,-0.140070,-0.100273
3,Santiago de Compostela,6293.0,44.316542,0.535516,0.969967,0.000477,0.000636,0.000477,0.000318,0.002225,...,0.001271,0.000000,0.000477,0.001112,0.004767,0.002066,Galicia,-0.103921,-0.140070,-0.100273
4,Vigo,25404.0,43.782436,0.526610,0.963116,0.000236,0.000197,0.002677,0.002047,0.003188,...,0.003858,0.000276,0.003188,0.000354,0.000630,0.001378,Galicia,-0.103921,-0.140070,-0.100273
5,Albacete,11912.0,39.921592,0.519476,0.966924,0.001763,0.002015,0.007220,0.004701,0.001259,...,0.000420,0.000923,0.000084,0.000588,0.001175,0.000588,Castilla-La Mancha,0.023379,-0.067050,0.004133
6,Alcala de Henares,17089.0,39.858740,0.508573,0.915384,0.003218,0.003043,0.006729,0.033530,0.001404,...,0.001404,0.001404,0.000351,0.001287,0.000176,0.001346,"Madrid, Comunidad de",-0.072231,-0.245129,-0.121979
7,Brunete,7043.0,37.869800,0.523215,0.915661,0.004969,0.009229,0.007667,0.003976,0.005963,...,0.000142,0.000000,0.000710,0.000426,0.001278,0.003266,"Madrid, Comunidad de",-0.072231,-0.245129,-0.121979
8,Colmenar Viejo,3403.0,38.691155,0.521011,0.919189,0.021746,0.008816,0.004702,0.006759,0.000882,...,0.001469,0.001469,0.000000,0.000294,0.004702,0.003820,"Madrid, Comunidad de",-0.072231,-0.245129,-0.121979
9,Fuenlabrada,41314.0,40.052161,0.512199,0.935712,0.009004,0.005906,0.008496,0.008762,0.001017,...,0.001138,0.001573,0.000363,0.000750,0.000411,0.000896,"Madrid, Comunidad de",-0.072231,-0.245129,-0.121979


In [46]:
collapsedAux.isna().sum()

closest_glovo_city            0
total_observations            0
mean_age                      0
mean_female_share             0
share_españa                  0
share_ecuador                 0
share_marruecos               0
share_colombia                0
share_rumanía                 0
share_argentina               0
share_perú                    0
share_francia                 0
share_bolivia                 0
share_alemania                0
share_venezuela               0
share_república_dominicana    0
share_cuba                    0
share_reino_unido             0
share_brasil                  0
share_china                   0
share_uruguay                 0
share_chile                   0
share_paraguay                0
share_italia                  0
autonomous_community          0
index_antiimmigration         0
index_xenophobia              0
index_all                     0
dtype: int64

In [47]:
collapsedAux.autonomous_community.value_counts(dropna=False)

Andalucía                      14
Madrid, Comunidad de           11
Cataluña                       10
Comunitat Valenciana            7
Galicia                         5
Castilla y León                 5
País Vasco                      5
Murcia, Región de               4
Asturias, Principado de         2
Balears, Illes                  2
Canarias                        2
Castilla-La Mancha              1
Extremadura                     1
Navarra, Comunidad Foral de     1
Cantabria                       1
Aragón                          1
Name: autonomous_community, dtype: int64

In [48]:
collapsedAux.to_excel('collapsed_aux.xlsx')

## glovo  

### collapse glovo orders by city, week, culinary culture

In [49]:
glovo = pd.read_csv('../outputs/restaurant_aggregation_spain.csv', index_col=0).rename(columns={'culinary_origin': 'culinary_culture', 'city': 'city_code'})
glovo

,year,week,store_name,store_address_id,city_code,culinary_culture,store_tag,orders_this_restaurant_this_week,orders_this_city_this_week,date
0,2016,6,Carrefour,105.0,BCN,other,Groceries,2,10,2016-02-12
1,2016,6,Chivuo's,35.0,BCN,US,Burger,1,10,2016-02-12
2,2016,6,Chok Barcelona,75.0,BCN,NaN,NaN,2,10,2016-02-12
3,2016,6,Comaxurros,84.0,BCN,NaN,NaN,1,10,2016-02-12
4,2016,6,Greenshots,72.0,BCN,NaN,NaN,1,10,2016-02-12
5,2016,6,Parking Pizza,71.0,BCN,Italy,Pizza & Italian,1,10,2016-02-12
6,2016,6,Pedidos Especiales Parafarmacia,3.0,BCN,NaN,NaN,1,10,2016-02-12
7,2016,6,Petit Bangkok,82.0,BCN,Thailand,Thai,1,10,2016-02-12
8,2016,7,Carrefour,108.0,BCN,other,Groceries,1,18,2016-02-19
9,2016,7,Carrefour,109.0,BCN,other,Groceries,1,18,2016-02-19


In [50]:
collapsedGlovo = (
    glovo
    .drop(columns=['store_address_id'])
    .groupby(['year', 'week', 'city_code', 'culinary_culture'])
    .agg({
        'orders_this_restaurant_this_week': 'sum', 
        'orders_this_city_this_week':       'first', # since this aggregation (city + culinary_culture) is MORE specific than just (city), its fine to take the first value
    })
    .reset_index()
)

collapsedGlovo

,year,week,city_code,culinary_culture,orders_this_restaurant_this_week,orders_this_city_this_week
0,2016,6,BCN,Italy,1,10
1,2016,6,BCN,Thailand,1,10
2,2016,6,BCN,US,1,10
3,2016,6,BCN,other,2,10
4,2016,7,BCN,Thailand,1,18
5,2016,7,BCN,US,1,18
6,2016,7,BCN,other,8,18
7,2016,7,MAD,Japan,3,8
8,2016,7,MAD,US,1,8
9,2016,7,MAD,other,2,8


### decode city_code

In [51]:
glovoCityCodes = pd.read_excel('glovo_city_codes.xlsx')
glovoCityCodes

,city_code,city_name
0,AGU,Aguadulce
1,ALB,Albacete
2,ALC,Alicante
3,ALG,Algeciras
4,ALM,Almeria
5,ALR,Alcantarilla
6,AVL,Avila
7,BCN,Barcelona
8,BIL,Bilbao
9,BJZ,Badajoz


In [52]:
collapsedGlovo = pd.merge(collapsedGlovo, glovoCityCodes, on='city_code', how='left')
collapsedGlovo

,year,week,city_code,culinary_culture,orders_this_restaurant_this_week,orders_this_city_this_week,city_name
0,2016,6,BCN,Italy,1,10,Barcelona
1,2016,6,BCN,Thailand,1,10,Barcelona
2,2016,6,BCN,US,1,10,Barcelona
3,2016,6,BCN,other,2,10,Barcelona
4,2016,7,BCN,Thailand,1,18,Barcelona
5,2016,7,BCN,US,1,18,Barcelona
6,2016,7,BCN,other,8,18,Barcelona
7,2016,7,MAD,Japan,3,8,Madrid
8,2016,7,MAD,US,1,8,Madrid
9,2016,7,MAD,other,2,8,Madrid


In [53]:
collapsedGlovo.city_name.isna().sum()

0

In [54]:
collapsedGlovo.to_excel('collapsed_glovo.xlsx')

### merge glovo to aux

In [55]:
collapsedAux = pd.read_excel('collapsed_aux.xlsx', index_col=0)
collapsedAux

,closest_glovo_city,total_observations,mean_age,mean_female_share,share_españa,share_ecuador,share_marruecos,share_colombia,share_rumanía,share_argentina,...,share_brasil,share_china,share_uruguay,share_chile,share_paraguay,share_italia,autonomous_community,index_antiimmigration,index_xenophobia,index_all
0,A Coruña,23636,43.946691,0.528685,0.967084,0.000381,0.000804,0.002919,0.000296,0.002750,...,0.003215,0.000465,0.003046,0.000465,0.000169,0.001777,Galicia,-0.103921,-0.140070,-0.100273
1,Ourense,6888,45.570412,0.539925,0.970383,0.000145,0.001597,0.004646,0.000726,0.001887,...,0.002613,0.000290,0.000145,0.000000,0.000871,0.000581,Galicia,-0.103921,-0.140070,-0.100273
2,Pontevedra,5121,41.683460,0.534075,0.968366,0.000586,0.003320,0.003710,0.000781,0.001562,...,0.003905,0.002343,0.001172,0.000000,0.000000,0.000391,Galicia,-0.103921,-0.140070,-0.100273
3,Santiago de Compostela,6293,44.316542,0.535516,0.969967,0.000477,0.000636,0.000477,0.000318,0.002225,...,0.001271,0.000000,0.000477,0.001112,0.004767,0.002066,Galicia,-0.103921,-0.140070,-0.100273
4,Vigo,25404,43.782436,0.526610,0.963116,0.000236,0.000197,0.002677,0.002047,0.003188,...,0.003858,0.000276,0.003188,0.000354,0.000630,0.001378,Galicia,-0.103921,-0.140070,-0.100273
5,Albacete,11912,39.921592,0.519476,0.966924,0.001763,0.002015,0.007220,0.004701,0.001259,...,0.000420,0.000923,0.000084,0.000588,0.001175,0.000588,Castilla-La Mancha,0.023379,-0.067050,0.004133
6,Alcala de Henares,17089,39.858740,0.508573,0.915384,0.003218,0.003043,0.006729,0.033530,0.001404,...,0.001404,0.001404,0.000351,0.001287,0.000176,0.001346,"Madrid, Comunidad de",-0.072231,-0.245129,-0.121979
7,Brunete,7043,37.869800,0.523215,0.915661,0.004969,0.009229,0.007667,0.003976,0.005963,...,0.000142,0.000000,0.000710,0.000426,0.001278,0.003266,"Madrid, Comunidad de",-0.072231,-0.245129,-0.121979
8,Colmenar Viejo,3403,38.691155,0.521011,0.919189,0.021746,0.008816,0.004702,0.006759,0.000882,...,0.001469,0.001469,0.000000,0.000294,0.004702,0.003820,"Madrid, Comunidad de",-0.072231,-0.245129,-0.121979
9,Fuenlabrada,41314,40.052161,0.512199,0.935712,0.009004,0.005906,0.008496,0.008762,0.001017,...,0.001138,0.001573,0.000363,0.000750,0.000411,0.000896,"Madrid, Comunidad de",-0.072231,-0.245129,-0.121979


In [56]:
collapsed = pd.merge(collapsedGlovo, collapsedAux, left_on='city_name', right_on='closest_glovo_city', how='left').drop(columns='closest_glovo_city')
collapsed

,year,week,city_code,culinary_culture,orders_this_restaurant_this_week,orders_this_city_this_week,city_name,total_observations,mean_age,mean_female_share,...,share_brasil,share_china,share_uruguay,share_chile,share_paraguay,share_italia,autonomous_community,index_antiimmigration,index_xenophobia,index_all
0,2016,6,BCN,Italy,1,10,Barcelona,142585.0,45.374100,0.532707,...,0.001957,0.001943,0.000982,0.001781,0.001101,0.005898,Cataluña,-0.040758,-0.129260,-0.065620
1,2016,6,BCN,Thailand,1,10,Barcelona,142585.0,45.374100,0.532707,...,0.001957,0.001943,0.000982,0.001781,0.001101,0.005898,Cataluña,-0.040758,-0.129260,-0.065620
2,2016,6,BCN,US,1,10,Barcelona,142585.0,45.374100,0.532707,...,0.001957,0.001943,0.000982,0.001781,0.001101,0.005898,Cataluña,-0.040758,-0.129260,-0.065620
3,2016,6,BCN,other,2,10,Barcelona,142585.0,45.374100,0.532707,...,0.001957,0.001943,0.000982,0.001781,0.001101,0.005898,Cataluña,-0.040758,-0.129260,-0.065620
4,2016,7,BCN,Thailand,1,18,Barcelona,142585.0,45.374100,0.532707,...,0.001957,0.001943,0.000982,0.001781,0.001101,0.005898,Cataluña,-0.040758,-0.129260,-0.065620
5,2016,7,BCN,US,1,18,Barcelona,142585.0,45.374100,0.532707,...,0.001957,0.001943,0.000982,0.001781,0.001101,0.005898,Cataluña,-0.040758,-0.129260,-0.065620
6,2016,7,BCN,other,8,18,Barcelona,142585.0,45.374100,0.532707,...,0.001957,0.001943,0.000982,0.001781,0.001101,0.005898,Cataluña,-0.040758,-0.129260,-0.065620
7,2016,7,MAD,Japan,3,8,Madrid,232122.0,44.400561,0.537597,...,0.001460,0.001874,0.000383,0.000999,0.002322,0.002921,"Madrid, Comunidad de",-0.072231,-0.245129,-0.121979
8,2016,7,MAD,US,1,8,Madrid,232122.0,44.400561,0.537597,...,0.001460,0.001874,0.000383,0.000999,0.002322,0.002921,"Madrid, Comunidad de",-0.072231,-0.245129,-0.121979
9,2016,7,MAD,other,2,8,Madrid,232122.0,44.400561,0.537597,...,0.001460,0.001874,0.000383,0.000999,0.002322,0.002921,"Madrid, Comunidad de",-0.072231,-0.245129,-0.121979


In [57]:
collapsed = collapsed.rename(columns={'orders_this_restaurant_this_week': 'orders_this_culture_this_week'})

In [58]:
# check that all city_codes only have one unique city_name
for cityCode in collapsed.city_code.unique():
    assert len(collapsed[collapsed.city_code == cityCode].city_name.unique()) == 1
print('check passed')


check passed


In [59]:
collapsed = collapsed.dropna(subset=['total_observations', 'index_all'], how='all').reset_index(drop=True)
collapsed

,year,week,city_code,culinary_culture,orders_this_culture_this_week,orders_this_city_this_week,city_name,total_observations,mean_age,mean_female_share,...,share_brasil,share_china,share_uruguay,share_chile,share_paraguay,share_italia,autonomous_community,index_antiimmigration,index_xenophobia,index_all
0,2016,6,BCN,Italy,1,10,Barcelona,142585.0,45.374100,0.532707,...,0.001957,0.001943,0.000982,0.001781,0.001101,0.005898,Cataluña,-0.040758,-0.129260,-0.065620
1,2016,6,BCN,Thailand,1,10,Barcelona,142585.0,45.374100,0.532707,...,0.001957,0.001943,0.000982,0.001781,0.001101,0.005898,Cataluña,-0.040758,-0.129260,-0.065620
2,2016,6,BCN,US,1,10,Barcelona,142585.0,45.374100,0.532707,...,0.001957,0.001943,0.000982,0.001781,0.001101,0.005898,Cataluña,-0.040758,-0.129260,-0.065620
3,2016,6,BCN,other,2,10,Barcelona,142585.0,45.374100,0.532707,...,0.001957,0.001943,0.000982,0.001781,0.001101,0.005898,Cataluña,-0.040758,-0.129260,-0.065620
4,2016,7,BCN,Thailand,1,18,Barcelona,142585.0,45.374100,0.532707,...,0.001957,0.001943,0.000982,0.001781,0.001101,0.005898,Cataluña,-0.040758,-0.129260,-0.065620
5,2016,7,BCN,US,1,18,Barcelona,142585.0,45.374100,0.532707,...,0.001957,0.001943,0.000982,0.001781,0.001101,0.005898,Cataluña,-0.040758,-0.129260,-0.065620
6,2016,7,BCN,other,8,18,Barcelona,142585.0,45.374100,0.532707,...,0.001957,0.001943,0.000982,0.001781,0.001101,0.005898,Cataluña,-0.040758,-0.129260,-0.065620
7,2016,7,MAD,Japan,3,8,Madrid,232122.0,44.400561,0.537597,...,0.001460,0.001874,0.000383,0.000999,0.002322,0.002921,"Madrid, Comunidad de",-0.072231,-0.245129,-0.121979
8,2016,7,MAD,US,1,8,Madrid,232122.0,44.400561,0.537597,...,0.001460,0.001874,0.000383,0.000999,0.002322,0.002921,"Madrid, Comunidad de",-0.072231,-0.245129,-0.121979
9,2016,7,MAD,other,2,8,Madrid,232122.0,44.400561,0.537597,...,0.001460,0.001874,0.000383,0.000999,0.002322,0.002921,"Madrid, Comunidad de",-0.072231,-0.245129,-0.121979


In [60]:
collapsed.to_excel('collapsed_all.xlsx') # to look at
collapsed.to_csv('collapsed_all.csv') # to use

## tests

In [61]:
collapsed[collapsed.index_all.isna()]

,year,week,city_code,culinary_culture,orders_this_culture_this_week,orders_this_city_this_week,city_name,total_observations,mean_age,mean_female_share,...,share_brasil,share_china,share_uruguay,share_chile,share_paraguay,share_italia,autonomous_community,index_antiimmigration,index_xenophobia,index_all


In [62]:
ass = pd.read_excel('association_table.xlsx', index_col=0)

In [63]:
ass[ass.closest_glovo_city == 'Laganes']

,municipality,autonomous_community,municipality_coords,closest_glovo_city
225,Alcalá de los Gazules,Andalucía,"[36.4618805, -5.7238891]",Laganes
